In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!rm -rf /kaggle/working/m2s
# Git clone your repository
!git clone https://github.com/namirahrasul/M2S.git /kaggle/working/m2s

In [ ]:
!rm -rf /kaggle/working/miniconda
!wget -q https://repo.anaconda.com/miniconda/Miniconda3-py39_23.5.2-0-Linux-x86_64.sh -O miniconda.sh
!chmod +x miniconda.sh
!bash ./miniconda.sh -b -f -p /kaggle/working/miniconda
!rm miniconda.sh

# Verify Miniconda installation (check for python binary)
!ls -l /kaggle/working/miniconda/bin/python || echo "Python binary missing!"

# Ensure Conda binaries are executable
!chmod +x /kaggle/working/miniconda/bin/conda
!chmod +x /kaggle/working/miniconda/bin/python  # Explicitly make python executable
!chmod +x /kaggle/working/miniconda/etc/profile.d/conda.sh

In [ ]:
os.environ['PATH'] = "/kaggle/working/miniconda/bin:" + os.environ['PATH']

In [ ]:
%%capture
!conda install conda=25.1.1 -y

In [ ]:
%%capture
# Initialize Conda
!conda init bash

# Create Conda environment with Python 3.9
!conda create -n m2s python=3.9 -y

In [ ]:
# Activate the environment
!source /kaggle/working/miniconda/etc/profile.d/conda.sh && conda activate m2s

In [ ]:
%%capture
!conda install -n m2s mpi4py openmpi -y
!conda install -n m2s git -y
!pip install gdown

In [ ]:
%%capture
!source /kaggle/working/miniconda/etc/profile.d/conda.sh && conda activate m2s && \
cd /kaggle/working/m2s && python setup.py

In [ ]:
!mkdir -p /kaggle/working/m2s/logs
!mkdir -p /kaggle/working/m2s/checkpoints

In [ ]:
# Download checkpoints from Google Drive (replace with your file IDs)
import gdown
#imagenet(1M epoch)
file_id_64 = '1OsL6UrYFUtmwBLcOdRViRcfTSdoIySjR'  
file_id_256 = '1fX0mcIWPAz7fd7tIWcVxFuPf65bqCjj3' 
gdown.download(f'https://drive.google.com/uc?id={file_id_64}', '/kaggle/working/m2s/checkpoints/imagenet64_1000000.pt', quiet=False)
gdown.download(f'https://drive.google.com/uc?id={file_id_256}', '/kaggle/working/m2s/checkpoints/imagenet256_1000000.pt', quiet=False)
#celeba-hq(1M epoch)
file_id_64 = '1GWdEYCqcGd55Gqkli80Z_O_t9cKuJWWP'  
file_id_256 = '16Hz_Fis2PPmq2RrFy9Kj9WkZoXdsc3K0' 
gdown.download(f'https://drive.google.com/uc?id={file_id_64}', '/kaggle/working/m2s/checkpoints/celebahq64_1000000.pt', quiet=False)
gdown.download(f'https://drive.google.com/uc?id={file_id_256}', '/kaggle/working/m2s/checkpoints/celebahq256_1000000.pt', quiet=False)

In [ ]:
run='''
CF_RESAMPLE_FLAGS="--t_T 250 --n_sample 1 --t_T_fine 75 --jump_length_coarse 20 --jump_length_fine 10 --jump_n_sample_coarse 10 --jump_n_sample_fine 10 --jump_interval_coarse 15 --jump_interval_fine 10"
BASELINE_RESAMPLE_FLAGS="--jump_length 10 --jump_n_sample 10"
MODEL_FLAGS="--num_heads 1 --attention_resolutions 16 --diffusion_steps 1000 --dropout 0.0 --image_size 256 --learn_sigma True --noise_schedule linear --num_channels 128 --num_head_channels 64 --num_res_blocks 1 --resblock_updown True --use_fp16 False --use_scale_shift_norm True"
SAMPLE_FLAGS="--timestep_respacing 270 --ddim_stride 5 --range_t 20 --batch_size 1"
BASIC_CONFIG_FLAGS="--use_inverse_masks False --use_cf True --use_ddim True --special_mask False" 
INPUT_PATH="--base_samples /kaggle/working/m2s/demo_split/celebahq/val --mask_path /kaggle/working/m2s/demo_split/mask/thin"
MODEL_PATH="--model_path_64 /kaggle/working/m2s/checkpoints/celebahq64_1000000.pt --model_path_256 /kaggle/working/m2s/checkpoints/celebahq256_1000000.pt"
OUTPUT_PATH="--save_dir /kaggle/working/m2s/results/celebahq/thick"

CUDA_VISIBLE_DEVICES=0 python sample.py $CF_RESAMPLE_FLAGS $BASELINE_RESAMPLE_FLAGS $CLASSIFIER_FLAGS $MODEL_FLAGS $SAMPLE_FLAGS $BASIC_CONFIG_FLAGS $INPUT_PATH $MODEL_PATH $CLASSIFIER_PATH $OUTPUT_PATH
'''
with open('/kaggle/working/m2s/run.sh', 'w') as f:
    f.write(run)

# Make it executable
!chmod +x /kaggle/working/m2s/run.sh

In [ ]:
!source /kaggle/working/miniconda/etc/profile.d/conda.sh && conda activate m2s && \
 cd /kaggle/working/m2s && ./run.sh

In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image
import math

def show_images_grid(save_dir):
    # Define the path to the outImg folder
    out_img_dir = save_dir
    
    # List all PNG files in the outImg directory
    image_files = sorted([f for f in os.listdir(out_img_dir) if f.endswith(".png")])
    print(f"Found {len(image_files)} images in {out_img_dir}")
    
    # Define the grid size (e.g., 2 rows x 3 columns for 6 images)
    num_images = len(image_files)
    cols = 3  # Number of columns in the grid
    rows = math.ceil(num_images / cols)  # Calculate rows needed
    
    # Create a figure and axes for the grid
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 4))
    
    # Flatten axes array for easy iteration (handles both 1D and 2D cases)
    axes = axes.flatten() if rows > 1 else [axes] if cols == 1 else axes
    
    # Load and plot each image
    for idx, image_file in enumerate(image_files):
        if idx >= len(axes):  # Stop if we run out of grid spaces
            break
        # Load the image
        image_path = os.path.join(out_img_dir, image_file)
        img = Image.open(image_path)
        
        # Display the image in the grid
        axes[idx].imshow(img)
        axes[idx].set_title(image_file, fontsize=10)
        axes[idx].axis("off")  # Hide axes for cleaner display
    
    # Hide any unused subplots
    for idx in range(len(image_files), len(axes)):
        axes[idx].axis("off")
    
    # Adjust layout and display the grid
    plt.tight_layout()
    plt.show()

In [ ]:
fid_str = '# === Metric Imports ===\n'\
'from lpips import LPIPS\n'\
'import torch\n'\
'import torch.nn.functional as F\n'\
'from math import log10\n'\
'from torchmetrics.functional.image import structural_similarity_index_measure\n'\
'from PIL import Image\n'\
'from torchvision import transforms\n'\
'import os\n'\
'import sys\n'\
'import io\n'\
'import subprocess\n'\
'# === Set Paths ===\n'\
'gt_dir = \'/kaggle/working/m2s/results/celebahq/thick/gtImg\'\n'\
'out_dir = \'/kaggle/working/m2s/results/celebahq/thick/outImg\'\n'\
'output_path = "/kaggle/working/inpainting_metrics_report.txt"\n'\
'# === Redirect Print to File ===\n'\
'f = open(output_path, "w")\n'\
'sys.stdout = io.TextIOWrapper(open(output_path, \'wb\'))\n'\
'# === Preprocessing ===\n'\
'filenames = sorted([\n'\
'    f for f in os.listdir(out_dir)\n'\
'    if f.lower().endswith((\'.png\', \'.jpg\', \'.jpeg\'))\n'\
'])\n'\
'print(f" Found {len(filenames)} image pairs to evaluate.\\n")\n'\
'# === FID Calculation (CLI) ===\n'\
'print("\\n Running FID Calculation using pytorch-fid...")\n'\
'try:\n'\
'    result = subprocess.run(\n'\
'        ["python", "-m", "pytorch_fid", gt_dir, out_dir],\n'\
'        capture_output=True, text=True\n'\
'    )\n'\
'    if result.returncode == 0:\n'\
'        fid_output = result.stdout.strip()\n'\
'        print(f"FID   : {fid_output.splitlines()[-1]} ↓ (lower is better)")\n'\
'    else:\n'\
'        print(" FID calculation failed:", result.stderr.strip())\n'\
'except Exception as e:\n'\
'    print(f" FID subprocess error: {e}")\n'\
'# === Restore Output ===\n'\
'sys.stdout = sys.__stdout__\n'\
'print(f"\\n Report saved to: {output_path}")\n'
with open('/kaggle/working/m2s/fid.py', 'w') as f:
    f.write(fid_str)
print("Successfully wrote fid.py to /kaggle/working/m2s/fid.py")

In [ ]:
!source /kaggle/working/miniconda/etc/profile.d/conda.sh && conda activate m2s && \
cd /kaggle/working/m2s && python fid.py

In [ ]:
import os
import zipfile
from pathlib import Path

# Define the directory containing the images (Kaggle path)
image_dir = '/kaggle/working/m2s/results/celebahq'

# Output path for the ZIP file (in Kaggle's writable directory)
output_dir = '/kaggle/working'
zip_name = os.path.join(output_dir, f'm2s_250_75_cfs_celebahq_thin.zip')

# Function to collect all image files
def collect_images(directory):
    image_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.txt')  # Add more extensions if needed
    image_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(image_extensions):
                image_files.append(os.path.join(root, file))
    return image_files

# Function to zip images
def zip_images(image_files, zip_path):
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file in image_files:
            zipf.write(file, os.path.relpath(file, image_dir))
    print(f"Images zipped as {zip_path}")

image_files = collect_images(image_dir)
print(f"Found {len(image_files)} images in {image_dir}")
zip_images(image_files, zip_name)
print(f"ZIP file created at {zip_name}. Download it from the Kaggle output panel.")